In [58]:
from datasets import load_dataset
dataset = load_dataset("Open-Orca/OpenOrca")

Found cached dataset parquet (/root/.cache/huggingface/datasets/Open-Orca___parquet/Open-Orca--OpenOrca-2065d6f0dc25b814/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 1/1 [00:00<00:00,  5.78it/s]


In [59]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 4233923
    })
})

In [60]:
dataset['train'][0]

{'id': 'niv.242684',
 'system_prompt': '',
 'question': "You will be given a definition of a task first, then some input of the task.\nThis task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.\n\nAFC Ajax (amateurs)'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.\nOutput:",
 'response': '[\n  ["AFC Ajax (amateurs)", "has ground", "Sportpark De Toekomst"],\n  ["Ajax Youth Academy", "plays at", "Sportpark De Toekomst"]\n]'}

In [61]:
from tqdm import tqdm

# Get the total number of examples
total = len(dataset['train'])

# Create a progress bar
pbar = tqdm(total=total)

def convert_format(example):
    # Update the progress bar
    pbar.update()

    # Creating new dictionary for new format
    new_dict = {}

    # Generating text value for new format
    text = '### 시스템: ' + example['system_prompt'] + '### 질문: ' + example['question'] + '\n\n' + '### 답변: ' + example['response'] + '<|endoftext|>'

    # Adding text value to new dictionary
    new_dict['text'] = text

    return new_dict

# Using map function to apply the processing function on all examples in parallel
new_format_dataset = dataset['train'].map(convert_format, load_from_cache_file=False)

# Close the progress bar
pbar.close()


100%|██████████| 4233923/4233923 [07:04<00:00, 9972.08it/s] 


In [62]:
new_format_dataset

Dataset({
    features: ['id', 'system_prompt', 'question', 'response', 'text'],
    num_rows: 4233923
})

In [63]:
new_format_dataset['text'][0]

'### 시스템: ### 질문: You will be given a definition of a task first, then some input of the task.\nThis task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.\n\nAFC Ajax (amateurs)\'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.\nOutput:\n\n### 답변: [\n  ["AFC Ajax (amateurs)", "has ground", "Sportpark De Toekomst"],\n  ["Ajax Youth Academy", "plays at", "Sportpark De Toekomst"]\n]<|endoftext|>'

In [64]:
import json
from tqdm import tqdm

# Assuming new_format_dataset is a list of dictionaries with a key 'text'
json_list = [json.dumps({"text": example['text']}) for example in new_format_dataset]

# Save the list to a JSON file
with open('../../datasets/raw_datasets/open_orca.json', 'w', encoding='utf-8') as f:
    # Create a new progress bar for the file writing
    pbar = tqdm(total=len(json_list))

    for json_str in json_list:
        f.write(json_str + '\n')  # Write JSON string and a newline character
        # Update the progress bar
        pbar.update()

    # Close the progress bar
    pbar.close()

100%|██████████| 4233923/4233923 [00:20<00:00, 209814.67it/s]


In [65]:
import json

with open('../../datasets/raw_datasets/open_orca.json', 'r', encoding='utf-8') as f:
    for line in f:
        print(json.loads(line)['text'])
        #print(translator.translate(json.loads(line)['text'], src="ko", dest="en"))
        break

### 시스템: ### 질문: You will be given a definition of a task first, then some input of the task.
This task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.

AFC Ajax (amateurs)'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.
Output:

### 답변: [
  ["AFC Ajax (amateurs)", "has ground", "Sportpark De Toekomst"],
  ["Ajax Youth Academy", "plays at", "Sportpark De Toekomst"]
]<|endoftext|>


In [37]:
import json

with open('../../datasets/KoAlpaca_v1.1a_textonly.json', 'r', encoding='utf-8') as f:
    for line in f:
        print(line)
        break

{"text":"### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>"}

